In [20]:
import pandas as pd
import os
import seaborn as sns

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

import shutil

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime as dt

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias


In [21]:
ruta_bd = r"C:\docsProyectos\8.GESTVALT\zInsumos\Información Recibida\FGDB-SOLEDAD-11062024.gdb"
scrach_bd = r"C:\docsProyectos\8.GESTVALT\1.GIS\PRO\GESVALT\GESVALT.gdb"

In [22]:
arcpy.env.workspace = ruta_bd
arcpy.env.overwriteOutput = True


CAPA_ANALISIS = 'LC_TERRENO'
CAMPO_DISOLUCION = 'MANZANA_VE'

capa_manzana_r = 'manzana_from_terrenosrurales'
capa_manzana_u = 'manzana_from_terrenosurbanos'

query_terreno_rural = "ESPACIO_DE = 'TERRENO_RURAL'"
query_terreno_urbano = "ESPACIO_DE = 'TERRENO_URBANO'"

ruta_capa_manzana_terrenorural = os.path.join(scrach_bd,capa_manzana_r)
ruta_capa_manzana_terrenourbano = os.path.join(scrach_bd,capa_manzana_u)

for capa in arcpy.ListFeatureClasses(wild_card=CAPA_ANALISIS):
    arcpy.MakeFeatureLayer_management(in_features=capa, out_layer='lyr_manzana_from_terrenorural', where_clause=query_terreno_rural)
    arcpy.MakeFeatureLayer_management(in_features=capa, out_layer='lyr_manzana_from_terrenourbano', where_clause=query_terreno_urbano)

    arcpy.Dissolve_management(in_features='lyr_manzana_from_terrenorural', out_feature_class=ruta_capa_manzana_terrenorural, dissolve_field=CAMPO_DISOLUCION, multi_part='SINGLE_PART')
    arcpy.Dissolve_management(in_features='lyr_manzana_from_terrenourbano', out_feature_class=ruta_capa_manzana_terrenourbano, dissolve_field=CAMPO_DISOLUCION, multi_part='SINGLE_PART')

print("Finaliza Proceso")

Finaliza Proceso


In [23]:
ruta_sectores = r"C:\docsProyectos\8.GESTVALT\zInsumos\SECTORES\SECTORES.shp"

df_sectores = pd.DataFrame.spatial.from_featureclass(ruta_sectores)

seleccion_columnas_sectores = ['Catastro','SHAPE']

df_sectores = df_sectores[seleccion_columnas_sectores]

df_sectores['priorizacion'] = None

dict_sectorizacion = {'Sector 02 Rural':99,
    'Sector 04 Rural':99,
    'Sector 05 Rural':99,
    'Sector 06 Rural':99,
    'Sector 00 Urbano':99,
    'Sector 02 Urbano':1,
    'Sector 03 Urbano':99,
    'Sector 04 Urbano':99,
    'Sector 04 Urbano':99,
    'Sector 05 Urbano':99,
    'Sector 06 Urbano':1,
    'Sector 03 Rural':99
    }

for columna in df_sectores.columns:
    for llave, valor in dict_sectorizacion.items(): 
        if columna == 'Catastro':
            df_sectores.loc[df_sectores[columna] == llave, 'priorizacion'] = valor

nombre_fc_sectores = 'sector'
ruta_fc_sectores = os.path.join(scrach_bd, nombre_fc_sectores)

df_sectores.spatial.to_featureclass(location=ruta_fc_sectores)


'C:\\docsProyectos\\8.GESTVALT\\1.GIS\\PRO\\GESVALT\\GESVALT.gdb\\sector'

In [24]:
arcpy.env.workspace = scrach_bd

memoria_centroide_mzn = "in_memory/centroide_manzana"
memoria_joinspatial_mzn = "in_memory/joinspatial_manzana"
memoria_mzn_w_priorizacion ="in_memory/joinspatial_mzn_w_priorizacion"

dict_capas = {'urbana':[capa_manzana_u,'manzana_urbana_priorizacion'],                        
    'rural':[capa_manzana_r,'manzana_rural_priorizacion']
    }

estandariazacion_mnzpriorizacion = ['MANZANA_VE','catastro','priorizacion','SHAPE']

for capa in arcpy.ListFeatureClasses():
    for llave, valor in dict_capas.items():
        if capa == valor[0]:
            arcpy.FeatureToPoint_management(in_features=capa, out_feature_class=memoria_centroide_mzn, point_location='INSIDE')

            arcpy.SpatialJoin_analysis(target_features=memoria_centroide_mzn,
                join_features='sector',
                out_feature_class=memoria_joinspatial_mzn,
                match_option='INTERSECT'
                )
        
            arcpy.SpatialJoin_analysis(target_features=capa,
                join_features=memoria_joinspatial_mzn,
                out_feature_class=memoria_mzn_w_priorizacion,
                match_option='INTERSECT'
                )
            
            globals()[f'df_mnz{llave}_priorizacion'] = pd.DataFrame.spatial.from_featureclass(memoria_mzn_w_priorizacion)
            globals()[f'df_mnz{llave}_priorizacion'] = globals()[f'df_mnz{llave}_priorizacion'][estandariazacion_mnzpriorizacion]

            ruta_salida_mnz_priorizacion = os.path.join(arcpy.env.workspace,valor[1])

            globals()[f'df_mnz{llave}_priorizacion'].spatial.to_featureclass(location=ruta_salida_mnz_priorizacion)


In [25]:

ruta_puntos_muestra = r"C:\docsProyectos\8.GESTVALT\zInsumos\Actualizacion_Soledad.gdb\Puntos_Muestra\Puntos_Muestra_V1"
df_punto_muestra = pd.DataFrame.spatial.from_featureclass(ruta_puntos_muestra)

estandarizacion_punto_muestra = ['T_ID','MANZANA_VE','COMIENZO_V','TERRCNPN','TERRARJU','TERRARFI','SHAPE']

df_punto_muestra = df_punto_muestra[estandarizacion_punto_muestra]

df_punto_muestra['COMIENZO_V'] = df_punto_muestra['COMIENZO_V'].astype('datetime64[ns]')

NOMBRE_CAPA_PTOMUESTRA = 'poligonos_puntos_muestra'
RUTA_SALIDA_PTOMUESTRA = os.path.join(arcpy.env.workspace, NOMBRE_CAPA_PTOMUESTRA)

df_punto_muestra.spatial.to_featureclass(location=RUTA_SALIDA_PTOMUESTRA)

memoria_centroide_ptomuestra = "in_memory/centroide_ptomuestra"

lista_zona = ['rural','urbana']

for capa in arcpy.ListFeatureClasses():
    if capa == NOMBRE_CAPA_PTOMUESTRA:
        for zona in lista_zona:
            arcpy.FeatureToPoint_management(in_features=capa, out_feature_class=memoria_centroide_ptomuestra,point_location='INSIDE')
            arcpy.MakeFeatureLayer_management(in_features=memoria_centroide_ptomuestra, out_layer='lyr_centroide_ptomuestra')
            arcpy.MakeFeatureLayer_management(in_features=f'manzana_{zona}_priorizacion', out_layer=f'lyr_manzana_{zona}_priorizacion')
            arcpy.SelectLayerByLocation_management(in_layer=f'lyr_manzana_{zona}_priorizacion', overlap_type='INTERSECT', select_features=memoria_centroide_ptomuestra, selection_type='NEW_SELECTION')
            arcpy.conversion.ExportFeatures(in_features=f'lyr_manzana_{zona}_priorizacion', out_features=f'mzn_{zona}_priorizacion_w_ptomuestra')